# Exemplos de queries SQL integrados ao Python e Pandas
Usaremos dados de exemplo do site [MySQL Tutorial](http://www.mysqltutorial.org/mysql-sample-database.aspx).

Para recriar o banco de dados em SQLite, sem Python, basta executar o arquivo com os comandos SQL na linha de comando do sistema operacional:

```shell
sqlite3 database.db < sample-database-dump.sql
```

O arquivo `database.db` será criado e tabelas e dados conforme especificados em `sql-sample-database.sql` serão populados.

Se estivéssemos lidando com um SGBD (sistema gerenciador de banco de dados) mais robusto, como MariaDB/MySQL, Oracle ou DB2, o comando acima seria diferente e deverá conter:

* o hostname ou IP do servidor de banco de dados
* usuário e senha de acesso
* nome do banco de dados em que vamos operar

Mas SQLite é bem mais simples e didático e opera num arquivo local.

## Visualizando o banco de dados, suas tabelas e dados

<img src="MySQL-Sample-Database-Schema.png">

Cada SGBD tem seu próprio ferramental para navegar nos dados. MariaDB/MySQL tem o popular [PHP MyAdmin](https://www.phpmyadmin.net), Oracle e DB2 tem suas próprias ferramentas proprietárias, SQLite tem [DB Browser for SQLite](https://sqlitebrowser.org) e uma ferramenta online chamada [SQLite Online](https://sqliteonline.com).

Para navegar nos dados do DB SQLite que acabamos de criar, use uma das 3 opções:

* Use o comando `sqlite3`.
* Instale o **DB Browser for SQLite** e abra o arquivo.
* Envie o arquivo para o site **SQLiteOnline.com**.

## SQLite

#### Vamos primeiro usar a estrutura e dados do arquivo `sql-sample-database.sql` para criar um banco de dados no arquivo `database.db`.

#### Usamos o método [`connect()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection) para criar um objeto de conexão com esse banco de dados.

In [ ]:
import pandas as pd
import sqlite3

db = sqlite3.connect('database.db')

#### Depois de ter um objeto de conexão, podemos criar um [`objeto Cursor`](https://docs.python.org/2.5/lib/sqlite3-Cursor-Objects.html) [`cursos()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor) e chamar seu método [`execute()`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.execute) para executar comandos `SQL`.

In [ ]:
script = 'sample-database-dump.sql'

db.cursor().executescript(open(script).read())

#### Camos criar uma conexão com a nossa base de dados.

In [ ]:
conn = sqlite3.connect('database.db')
curs = conn.cursor()
#conn.close()

In [ ]:
query = """CREATE TABLE IF NOT EXISTS countries (

key INTEGER PRIMARY KEY AUTOINCREMENT,

name text UNIQUE,

founding_year INTEGER,

capital text

);"""

#query = """DROP TABLE countries ;"""

curs.execute(query)

conn.commit()

#### Vamos criar agora uma tabela.

In [ ]:
#query = "DELETE FROM countries WHERE name == 'BRASIL';"

query  = """INSERT INTO countries (key, name, founding_year, capital)
    VALUES (1, 'BRASIL', 1500, 'SALVADOR')
;"""

conn.commit()

curs.execute(query)

#### Vamos observar a tabela que foi criada.

In [ ]:
query = """SELECT * 
    FROM countries
;"""

queryResult = pd.read_sql_query(query, db)

queryResult.head()

In [ ]:
res = curs.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
for name in res:
    print (name[0])

#### Vamos inserir valores na tabela criada.

In [ ]:
#curs.execute("INSERT INTO countries (key, name, founding_year, capital) VALUES (1, 'BRASIL', 1500, 'SALVADOR');")
#conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query, db)

queryResult.head()

#### Vamos inserir mais alguns países.

In [ ]:

curs.execute("""INSERT INTO countries (key, name, founding_year, capital)

VALUES 
(2, 'MÉXICO', 1519, 'CIDADE DO MÉXICO'),
(3, 'ARGENTINA', 1516, 'BUENOS AIRES')
;""")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### Vamos inserir o país "EUA".

In [ ]:
curs.execute("INSERT INTO countries (key, name, founding_year, capital) VALUES (4, 'EUA', 1585, 'St. Augustine');")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### Inserimos também o país "VENEZUELA".

In [ ]:
curs.execute("""INSERT INTO countries  

(name, founding_year, capital)

VALUES 

('VENEZUELA', 1519, 'CARACAS')
;""")

conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### Podemos também fazer uma atualização de valores na tabela criada.

In [ ]:
curs.execute("""UPDATE countries  
SET capital = 'BRASILIA'

WHERE
name = 'BRASIL' AND founding_year = 1500

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### É possível ainda excluir valores.

In [ ]:

curs.execute("""UPDATE countries  

SET capital = NULL

WHERE

name = 'ARGENTINA'

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### Ou também excluir uma linha inteira.

In [ ]:
curs.execute("""DELETE FROM countries  

WHERE
name = 'ARGENTINA'

;""")
conn.commit()

query = "SELECT * FROM countries;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### Para excluir uma tabela, usamos o comando [`DROP TABLE`](https://www.sqlitetutorial.net/sqlite-drop-table/).

<img src="RobertDROPTABLEStudants.png">

In [ ]:

curs.execute("""
DROP TABLE countries;""")

conn.commit()

In [ ]:
res = conn.execute("""SELECT name 
FROM sqlite_master 
WHERE type = 'table'
;"""
                  )
for name in res:
    print (name[0])

## Vamos focar em ler dados de um banco relacional utilizando SQL

#### A operação mais comum em um banco de dados é a leitura de dados e para isso precisamos escrever uma requisição, tmbém conhecido como `QUERY`.

Uma `query` é construída em etapas:

1. [`SELECT`](https://www.sqlitetutorial.net/sqlite-select/) = irá iniciar a seleção de dados que faremos a leitura

2. FROM = irá selecionar as tabelas de interesse

3. [`WHERE`](https://www.sqlitetutorial.net/sqlite-where/) = vamos declarar as condições para ler os dados

4. [`GROUP BY`](https://www.sqlitetutorial.net/sqlite-group-by/) = podemos agrupar algum dado considerando uma coluna 

5. Pós processamento = podemos ordenar ([`ORDER BY`](https://www.sqlitetutorial.net/sqlite-order-by/)) ou estabelecer limites ([`LIMIT`](https://www.sqlitetutorial.net/sqlite-limit/))



## Vamos construir aos poucos nossa query

In [ ]:
query = "SELECT * FROM customers;"

queryResult = pd.read_sql_query(query, db)

queryResult.head()

#### Vamos ler apenas algumas colunas específicas.

In [ ]:
query = "SELECT customerName, phone FROM customers;"

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### Vamos utilizar pela primeira vez uma condição para fazer a leitura.

In [ ]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country = 'USA' ;"""

query2 = pd.read_sql_query(query,db)

query2.head()

#### Agora uma condição composta.

In [ ]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country = 'USA' OR country = 'France';"""

query2 = pd.read_sql_query(query,db)

#query2.shape
query2.head()

#### Podemos usar o comando [`NOT IN`](https://www.ramosdainformatica.com.br/banco_de_dados/sql-not-in-ou-not-exists/)

In [ ]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country NOT IN ('USA', 'France');"""

query2 = pd.read_sql_query(query,db)

query2.shape
query2.head()

#### Uma terceira seleção composta.

In [ ]:
query = """SELECT * 
          
          FROM customers 
          
          WHERE country ='USA' AND city ='NYC';"""

query2 = pd.read_sql_query(query,db)

query2.shape

#### Para o caso de termos mais de uma tabela e quisermos ler apenas algumas colunas de cada tabela, utilzamos o [`JOIN`](https://www.sqlitetutorial.net/sqlite-join/).

In [ ]:
query = """SELECT
    o.priceEach,
    p.productCode,
    p.productName
    
    FROM 
    orderdetails AS o INNER JOIN products AS p
/*
    orderdetails AS o,products AS p    
*/
    
    WHERE
    o.productCode=p.productCode;"""

queryResult = pd.read_sql_query(query,db)

queryResult.shape
queryResult.head()

#### Para ordenar as linhas pelos valores de uma coluna em ordem decrescente devemos utilizar o comando [`DESC`](https://www.tutorialspoint.com/sqlite/sqlite_order_by.htm) após a coluna de referência.

In [ ]:

query = """SELECT
    o.priceEach,
    p.productCode,
    p.productName
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode = p.productCode
    
    ORDER BY
    o.priceEach DESC;"""

queryResult = pd.read_sql_query(query,db)

queryResult.shape
queryResult.head()

#### O comando [`GROUP BY`](https://www.sqlitetutorial.net/sqlite-group-by/) é utilizado para agrupar um conjunto de linhas em acordo com um determinado atributo comum. Se usado em conjunto com o[`Order by`](https://www.sqlitetutorial.net/sqlite-order-by/), deve-se lembrar que order by segue group by em [ordem de operação](https://learnsql.com/blog/sql-order-of-operations/#:~:text=Six%20Operations%20to%20Order%3A%20SELECT,developer%20to%20know%20this%20order.).

In [ ]:
query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode = p.productCode
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    VendaTotal;"""

queryResult = pd.read_sql_query(query,db)

queryResult.head()

#### Podemos realisar a contagem de valores com o comando  [`COUNT`](https://www.sqlitetutorial.net/sqlite-count-function/).

In [ ]:

query = """SELECT
    p.productVendor AS Vendedor,
    COUNT(p.productVendor) AS 'TotalRegistro'
    
    FROM 
    products AS p
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    TotalRegistro;"""

queryResult = pd.read_sql_query(query,db)

queryResult.head()

## Lista completa de agregações 

<img src="list_aggregation.png">

#### O comando [`HAVING`](https://www.sqlitetutorial.net/sqlite-having/) é usado para especificar uma condição de pesquisa para um grupo.

In [ ]:

query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o INNER JOIN products AS p
    
    WHERE
    o.productCode = p.productCode
    
    GROUP BY
    p.productVendor
    
    HAVING
    o.quantityOrdered < 30
    
    ORDER BY
    VendaTotal
    ;"""

queryResult = pd.read_sql_query(query,db)

queryResult

#### Podemos limitar a quantidade de linhas que serão lidas com o commando [`LIMIT`](https://www.sqlitetutorial.net/sqlite-limit/).

In [ ]:

query = """SELECT
    p.productVendor AS Vendedor,
    SUM(o.priceEach*o.quantityOrdered) AS 'VendaTotal'
    
    FROM 
    orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode
    
    GROUP BY
    p.productVendor
    
    ORDER BY
    VendaTotal DESC
    
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query,db)

queryResult

## SQLite com Pandas

###  <span style = "color:blue">Prática independente.</span>

#### Agora começamos a fazer queries:

## Lista de funcionários por chefe.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   employees e1,
   employees e2

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   e1.reportsTo = e2.employeeNumber

MOSTRE AS COLUNAS
   e2.firstName||' '||e2.lastName AS boss,
   e2.jobTitle                    AS department,
   e1.employeeNumber              AS ID,
   e1.firstName||' '||e1.lastName AS employee

ORDENE POR
   boss
```

####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
   e2.firstName||' '||e2.lastName AS boss,
   e2.jobTitle                    AS department,
   e1.employeeNumber              AS ID,
   e1.firstName||' '||e1.lastName AS employee    

    FROM 
    employees AS e1, employees AS e2

    ORDER BY boss 
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query, db)

queryResult
-->

## Inspeção de pedidos de compra.

#### Dados sobre pedidos aparecem nas tabelas `orders` e `orderdetails` e o que relaciona elas é a coluna `orderNumber`. Vamos inspecionar um único pedido.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   orderNumber=10103

MOSTRE AS COLUNAS
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal
    
ORDENE POR
   orderLineNumber
```

####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal,
    orderNumber

    FROM 
    orderdetails

    WHERE orderNumber = 10103
    
    ORDER BY orderLineNumber
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

#### Vamos melhorar a leitura dessa tabela adicionando a descrição do produto. Faremos um `JOIN` com a tabela `products`.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails AS o,
   products AS p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode AND
    orderNumber=10103

MOSTRE AS COLUNAS
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    
ORDENE POR
   orderLineNumber
```

####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    FROM 
    orderdetails AS o,
    products AS p
    
    WHERE o.productCode = p.productCode AND orderNumber = 10103
    
    ORDER BY orderLineNumber
    LIMIT
    3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

## Cálculo de totais e subtotais


#### Performance de venda por categoria de produtos.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productLine

ORDENE INVERSAMENTE POR
   lineTotal
```

####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

    FROM 
    orderdetails AS o, products AS p
    /*orderdetails AS o INNER JOIN products AS p*/
    
    GROUP BY p.productLine
    ORDER BY lineTotal
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

#### Qual é o produto que mais vende ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productName,
    count(o.quantityOrdered) AS nItems,
#    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productCode

ORDENE INVERSAMENTE POR
   lineTotal
```

####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT    
    p.productName,
    count(o.quantityOrdered) AS nItems,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

    FROM orderdetails o INNER JOIN products p
    
    WHERE o.productCode = p.productCode
    
    GROUP BY p.productCode
    
    ORDER BY lineTotal
    
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult

-->

#### Qual é o cliente que mais compra ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,
    customers as c

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber AND
    c.customerNumber = o.customerNumber

MOSTRE AS COLUNAS
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    c.customerNumber

ORDENE INVERSAMENTE POR
   c.customerNumber
```

####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal,
    sum(od.quantityOrdered) AS nItems
    
    FROM 
    orderdetails AS od INNER JOIN orders AS o INNER JOIN customers AS c

    WHERE od.orderNumber = o.orderNumber AND c.customerNumber = o.customerNumber

    GROUP BY c.customerNumber
    
    ORDER BY c.customerNumber
        
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult
-->

#### Vendas e valores por mês.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber

MOSTRE AS COLUNAS
    substr(o.orderDate,1,7) as month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    month

ORDENE INVERSAMENTE POR
   month
```


####  <span style = "color:red">Código original.</span>
<!--- 
query = """SELECT
    substr(o.orderDate,1,7) AS month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal,
    sum(od.quantityOrdered) AS nItems
    
    FROM orderdetails as od INNER JOIN orders as o
    
    WHERE od.orderNumber = o.orderNumber

    GROUP BY month
    
    ORDER BY month
        
    LIMIT 3;"""

queryResult = pd.read_sql_query(query, db)

queryResult
-->